# Elementary Symbol Tables and Binary Search Trees

A **symbol table** is a key-value pair abstraction where you can *insert* a value with a specified key, and given a key, *search* for the corresponding value.